In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
rrebirrth_animals_with_attributes_2_path = kagglehub.dataset_download('rrebirrth/animals-with-attributes-2')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Hayvan Türlerini Sınıflandırma Projesi

Bu proje, çeşitli hayvan türlerini sınıflandırmak için bir Convolutional Neural Network (CNN) modeli oluşturmayı amaçlamaktadır. Proje, Aygaz Görüntü İşleme Bootcamp kapsamında gerçekleştirilmiştir.

## Proje Adımları

### 1. Gerekli Kütüphanelerin İçe Aktarılması
```python
import os  # Dosya ve klasör yönetimi işlemleri için.
import numpy as np  # Sayısal hesaplamalar ve veri manipülasyonu için.
import shutil  # Dosya ve klasör yönetimi işlemleri için.
import random  # Rastgele işlemler için.
from glob import glob  # Dosya yollarını bulmak için.
import cv2  # Görüntü işleme ve boyutlandırma için.
import matplotlib.pyplot as plt  # Sonuçları görselleştirmek için.
from sklearn.model_selection import train_test_split  # Eğitim ve test veri setlerini ayırmak için.
from sklearn.preprocessing import LabelEncoder  # Etiketleri sayısal değerlere dönüştürmek için.
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Görüntü veri artırma (augmentation) için.
from tensorflow.keras.models import Sequential  # Model oluşturmak için.
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization  # Model katmanlarını oluşturmak için.
from tensorflow.keras.utils import to_categorical  # Etiketleri kategorik verilere dönüştürmek için.
from tensorflow import keras  # Derin öğrenme modelleme ve işlevsellik için.
from tensorflow.keras import models, layers  # Keras ile model ve katmanlar oluşturmak için.
import seaborn as sns  # İstatistiksel veri görselleştirmeleri için.
import plotly.express as px  # Etkileşimli ve dinamik grafikler oluşturmak için.
import missingno as msno  # Eksik veri analizi ve giderilmesi için.
import warnings  # Uyarıları kontrol etmek için.
warnings.filterwarnings("ignore")  # Tüm uyarıları gizlemek için.
from matplotlib import style  # Grafik stillerini değiştirmek için.
style.use("ggplot")  # ggplot stilini uygular.
from PIL import Image  # Görüntü işleme için.
from skimage import io, color, filters  # Scikit-Image modülleri.
import SimpleITK as sitk  # Tıbbi görüntü işleme için.


In [ ]:
import os  # Dosya ve klasör yönetimi işlemleri için.
import numpy as np  # Sayısal hesaplamalar ve veri manipülasyonu için.
import shutil  # Dosya ve klasör yönetimi işlemleri için.
import random  # Rastgele işlemler için.
from glob import glob  # Dosya yollarını bulmak için.
import cv2  # Görüntü işleme ve boyutlandırma için.
import matplotlib.pyplot as plt  # Sonuçları görselleştirmek için.
from sklearn.model_selection import train_test_split  # Eğitim ve test veri setlerini ayırmak için.
from sklearn.preprocessing import LabelEncoder  # Etiketleri sayısal değerlere dönüştürmek için.
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Görüntü veri artırma (augmentation) için.
from tensorflow.keras.models import Sequential  # Model oluşturmak için.
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization  # Model katmanlarını oluşturmak için.
from tensorflow.keras.utils import to_categorical  # Etiketleri kategorik verilere dönüştürmek için.
from tensorflow import keras  # Derin öğrenme modelleme ve işlevsellik için.
from tensorflow.keras import models, layers  # Keras ile model ve katmanlar oluşturmak için.
import seaborn as sns  # İstatistiksel veri görselleştirmeleri için.
import plotly.express as px  # Etkileşimli ve dinamik grafikler oluşturmak için.
import missingno as msno  # Eksik veri analizi ve giderilmesi için.
import warnings  # Uyarıları kontrol etmek için.
warnings.filterwarnings("ignore")  # Tüm uyarıları gizlemek için.
from matplotlib import style  # Grafik stillerini değiştirmek için.
style.use("ggplot")  # ggplot stilini uygular.
from PIL import Image  # Görüntü işleme için.
from skimage import io, color, filters  # Scikit-Image modülleri.
import SimpleITK as sitk  # Tıbbi görüntü işleme için.


In [ ]:
# Örnek olarak bir resim dosyasını okuyalım ve gösterelim
image_folder = '/kaggle/input/animals-with-attributes-2/Animals_with_Attributes2/JPEGImages'
image_path = os.path.join(image_folder, 'antelope', 'antelope_10001.jpg')  # Belirli bir resim dosyasını seçelim
image = cv2.imread(image_path)
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.axis('off')  # Eksenleri kapatalım
plt.show()


# **> 2. Veri Setini Kullanma**

Veri seti, Kaggle üzerinde bulunan "Animals with Attributes 2" veri setidir.

JPEGImages klasöründe 50 farklı hayvan sınıfı bulunmaktadır.

Projede yalnızca 10 sınıf kullanılacaktır: collie, dolphin, elephant, fox, moose, rabbit, sheep, squirrel, giant panda ve polar bear.

In [ ]:
# Veri seti klasörünün yolunu belirtelim
dataset_path = '/kaggle/input/animals-with-attributes-2/Animals_with_Attributes2/JPEGImages'

# Sınıfları belirlemek için klasör adlarını listeleyelim
classes = os.listdir(dataset_path)
print("Sınıflar:", classes)

# Her sınıftan bir örnek görüntü gösterelim
fig, axes = plt.subplots(len(classes) // 5 + 1, 5, figsize=(15, len(classes) * 1.5))
axes = axes.flatten()

for i, class_name in enumerate(classes):
    image_folder = os.path.join(dataset_path, class_name)
    image_files = os.listdir(image_folder)
    if image_files:
        image_path = os.path.join(image_folder, image_files[0])  # İlk görüntüyü alalım
        image = cv2.imread(image_path)
        axes[i].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        axes[i].set_title(class_name)
        axes[i].axis('off')

plt.tight_layout()
plt.show()


In [ ]:
# Veri seti klasörünün yolunu belirtelim
dataset_path = '/kaggle/input/animals-with-attributes-2/Animals_with_Attributes2/JPEGImages'
selected_classes = ['collie', 'dolphin', 'elephant', 'fox', 'moose', 'rabbit', 'sheep', 'squirrel', 'giant panda', 'polar bear']
output_path = '/kaggle/working/selected_images'

# Seçilen sınıfları ve her sınıftan 650 resmi yeni bir klasöre kopyalayalım
if not os.path.exists(output_path):
    os.makedirs(output_path)

for class_name in selected_classes:
    class_folder = os.path.join(dataset_path, class_name.replace(' ', '+'))
    output_class_folder = os.path.join(output_path, class_name.replace(' ', '+'))
    if not os.path.exists(output_class_folder):
        os.makedirs(output_class_folder)

    images = os.listdir(class_folder)[:650]  # İlk 650 resmi seçelim
    for image_name in images:
        image_path = os.path.join(class_folder, image_name)
        shutil.copy(image_path, output_class_folder)

# Resimleri aynı boyuta getirme ve normalize etme
image_size = (128, 128)  # İstediğiniz boyutu belirleyin
normalized_images = []

for class_name in selected_classes:
    class_folder = os.path.join(output_path, class_name.replace(' ', '+'))
    for image_name in os.listdir(class_folder):
        image_path = os.path.join(class_folder, image_name)
        image = cv2.imread(image_path)
        img_resized = cv2.resize(image, image_size)
        img_normalized = img_resized / 255.0
        normalized_images.append((img_normalized, class_name))

# Normalized_images listesi, normalize edilmiş resimleri ve sınıf adlarını içerir


In [ ]:
import matplotlib.pyplot as plt

# Her sınıftan bir örnek görüntü gösterelim
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
axes = axes.flatten()

for i, (img, class_name) in enumerate(normalized_images[:10]):  # İlk 10 sınıfı alalım
    axes[i].imshow(img)
    axes[i].set_title(class_name)
    axes[i].axis('off')

plt.tight_layout()
plt.show()


> # 3. Veri Setini Hazırlama
Her sınıftan yalnızca ilk 650 resim kullanılmıştır.

Tüm resimler 128x128 boyutuna getirilmiş ve normalize edilmiştir.

Veriler eğitim (%70) ve test (%30) olarak ikiye ayrılmıştır.

In [ ]:
# Veri seti klasörünün yolunu belirtelim
dataset_path = '/kaggle/input/animals-with-attributes-2/Animals_with_Attributes2/JPEGImages'
selected_classes = ['collie', 'dolphin', 'elephant', 'fox', 'moose', 'rabbit', 'sheep', 'squirrel', 'giant panda', 'polar bear']
output_path = '/kaggle/working/selected_images'

# Seçilen sınıfları ve her sınıftan 650 resmi yeni bir klasöre kopyalayalım
if not os.path.exists(output_path):
    os.makedirs(output_path)

for class_name in selected_classes:
    class_folder = os.path.join(dataset_path, class_name.replace(' ', '+'))
    output_class_folder = os.path.join(output_path, class_name.replace(' ', '+'))
    if not os.path.exists(output_class_folder):
        os.makedirs(output_class_folder)

    images = os.listdir(class_folder)[:650]  # İlk 650 resmi seçelim
    for image_name in images:
        image_path = os.path.join(class_folder, image_name)
        shutil.copy(image_path, output_class_folder)

# Resimleri aynı boyuta getirme ve normalize etme
image_size = (128, 128)  # İstediğiniz boyutu belirleyin
X = []
y = []

for class_name in selected_classes:
    class_folder = os.path.join(output_path, class_name.replace(' ', '+'))
    for image_name in os.listdir(class_folder):
        image_path = os.path.join(class_folder, image_name)
        image = cv2.imread(image_path)
        img_resized = cv2.resize(image, image_size)
        img_normalized = img_resized / 255.0
        X.append(img_normalized)
        y.append(class_name)

X = np.array(X)
y = np.array(y)

# Etiketleri sayısal değerlere dönüştürelim
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Etiketleri one-hot encoding yapalım
y = to_categorical(y)

# Veriyi eğitim ve test setlerine ayıralım
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print("Eğitim seti boyutu:", X_train.shape)
print("Test seti boyutu:", X_test.shape)


> # 3. CNN Modelinin Tasarlanması
10 hayvan sınıfını ayırt edecek bir CNN modeli tasarlanmıştır.

Modelde kullanılan katmanlar: Convolution, Activation, Pooling, Fully Connected, Dropout.

Kullanılan kayıp fonksiyonu: categorical_crossentropy.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Modeli tanımlayalım
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')  # 10 sınıf için
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Veri artırma tekniklerini tanımlayalım
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Eğitim setine veri artırma uygulayalım
train_datagen = datagen.flow(X_train, y_train, batch_size=32)

# Modeli eğitirken veri artırma kullanımı
model.fit(train_datagen, epochs=50, validation_data=(X_test, y_test))


In [ ]:
import matplotlib.pyplot as plt

# Eğitim setinden örnek görüntüler gösterelim
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
axes = axes.flatten()

for i in range(10):  # İlk 10 görüntüyü alalım
    axes[i].imshow(X_train[i])
    axes[i].set_title(label_encoder.inverse_transform([np.argmax(y_train[i])])[0])
    axes[i].axis('off')

plt.tight_layout()
plt.show()

# Test setinden örnek görüntüler gösterelim
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
axes = axes.flatten()

for i in range(10):  # İlk 10 görüntüyü alalım
    axes[i].imshow(X_test[i])
    axes[i].set_title(label_encoder.inverse_transform([np.argmax(y_test[i])])[0])
    axes[i].axis('off')

plt.tight_layout()
plt.show()


> # 4. Modelin Test Edilmesi
Model, test verisiyle test edilmiştir.

Modelin başarısı (accuracy) değerlendirilmiştir.

In [ ]:
# Modeli test edelim
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Doğruluk Oranı: {test_accuracy * 100:.2f}%")
print(f"Test Kayıp Değeri: {test_loss:.4f}")


> #     5. Tekrar eğitilir ve test edilir

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

# Modeli tanımlayalım
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    Conv2D(256, (3, 3), activation='relu'),  # Yeni konvolüsyon katmanı
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')  # 10 sınıf için
])

# Modeli derleyelim
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Modeli eğitelim
model.fit(train_datagen, epochs=50, validation_data=(X_test, y_test))


In [ ]:
# Modeli test edelim
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Doğruluk Oranı: {test_accuracy * 100:.2f}%")
print(f"Test Kayıp Değeri: {test_loss:.4f}")


> # 6. Resimlerin Kontrast Arttırma Ve Döndürme ile Manipüle Edilmesi

In [ ]:
def manipulate_images_v2(images):
    manipulated_images = []
    for img in images:
        manipulated = cv2.convertScaleAbs(img, alpha=2.0, beta=0)  # Kontrast artırma
        manipulated = cv2.rotate(manipulated, cv2.ROTATE_90_CLOCKWISE)  # Döndürme
        manipulated_images.append(manipulated)
    return np.array(manipulated_images)

# Test setini manipüle edelim
X_test_manipulated_v2 = manipulate_images_v2(X_test)

# Modeli manipüle edilmiş test seti ile değerlendirelim
manipulated_loss_v2, manipulated_accuracy_v2 = model.evaluate(X_test_manipulated_v2, y_test)
print(f"Manipüle Edilmiş Test Doğruluğu (Kontrast + Döndürme): {manipulated_accuracy_v2 * 100:.2f}%")
print(f"Manipüle Edilmiş Test Kayıp Değeri (Kontrast + Döndürme): {manipulated_loss_v2:.4f}")


> # 7. Manipüle Edilmiş Test Setine Renk Sabitliği Algoritması Uygulanması ve Denemesi

In [ ]:
def apply_color_correction(image):
    avg_b, avg_g, avg_r = cv2.mean(image)[:3]
    scale_b = 1.0 / (avg_b + 1e-5)
    scale_g = 1.0 / (avg_g + 1e-5)
    scale_r = 1.0 / (avg_r + 1e-5)
    corrected_image = image * np.array([scale_b, scale_g, scale_r])
    corrected_image = np.clip(corrected_image, 0, 255).astype(np.uint8)
    return corrected_image

# Manipüle edilmiş test setine renk sabitliği uygulayalım
X_test_corrected_v2 = np.array([apply_color_correction(img) for img in X_test_manipulated_v2])

# Modeli renk sabitliği uygulanmış test seti ile değerlendirelim
corrected_loss_v2, corrected_accuracy_v2 = model.evaluate(X_test_corrected_v2, y_test)
print(f"Renk Sabitliği Uygulanmış Test Doğruluğu (Manipüle Edilmiş): {corrected_accuracy_v2 * 100:.2f}%")
print(f"Renk Sabitliği Uygulanmış Test Kayıp Değeri (Manipüle Edilmiş): {corrected_loss_v2:.4f}")


> # 8. Farklı Test Setlerinin Başarılarını Karşılaştırma ve Raporlama

In [ ]:
# Orijinal test seti ile modeli değerlendirelim
loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Orijinal Test Seti Doğruluğu: {test_accuracy * 100:.2f}%")

# Manipüle edilmiş test seti ile modeli değerlendirelim
manipulated_loss_v2, manipulated_accuracy_v2 = model.evaluate(X_test_manipulated_v2, y_test)
print(f"Manipüle Edilmiş Test Doğruluğu (Kontrast + Döndürme): {manipulated_accuracy_v2 * 100:.2f}%")

# Renk sabitliği uygulanmış test seti ile modeli değerlendirelim
corrected_loss_v2, corrected_accuracy_v2 = model.evaluate(X_test_corrected_v2, y_test)
print(f"Renk Sabitliği Uygulanmış Test Doğruluğu (Manipüle Edilmiş): {corrected_accuracy_v2 * 100:.2f}%")

# Sonuçları yazdıralım
print("\nSonuçlar:")
print(f"Orijinal Test Seti Doğruluğu: {test_accuracy * 100:.2f}%")
print(f"Manipüle Edilmiş Test Seti Doğruluğu (Kontrast + Döndürme): {manipulated_accuracy_v2 * 100:.2f}%")
print(f"Renk Sabitliği Uygulanmış Test Seti (Manipüle Edilmiş): {corrected_accuracy_v2 * 100:.2f}%")
